In [75]:

## Читаем с автоматическим офсетом
from confluent_kafka import Consumer, KafkaException
# Конфигурация консюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my-group11wfewf12',
    'auto.offset.reset': 'earliest',  # Чтение с самого начала, если офсеты не найдены
    'enable.auto.commit': True        # Включение автоматического коммита офсетов
}
# Создание консюмера
consumer = Consumer(conf)
# Подписка на топик
consumer.subscribe(['my-avro-topic'])
try:
    for _ in range(5):
        msg = consumer.poll(timeout=1.0)  # Ожидание новых сообщений  
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        print(f'Received message: {msg.value().decode("utf-8")}')
        
finally:
    # Закрытие консюмера
    consumer.close()


Received message:     qwered 
Received message:     qwered 
Received message:     qwered 


## Читаем с РУЧНЫМ офсетом

In [76]:

from confluent_kafka import Consumer, KafkaException
# Конфигурация консюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my-group-222222222', ## другая группа
    'auto.offset.reset': 'earliest',  # Чтение с самого начала, если офсеты не найдены
    'enable.auto.commit': False        # 
}
# Создание консюмера
consumer = Consumer(conf)
# Подписка на топик
consumer.subscribe(['my-avro-topic'])
try:
    for _ in range(15):
        msg = consumer.poll(timeout=1.0)  # Ожидание новых сообщений  
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        print(f'Received message: {msg.value().decode("utf-8")}')
        
finally:
    # Закрытие консюмера
    consumer.close()

Received message:     qwered 
Received message:     qwered 
Received message:     qwered 
Received message:     qwered 
Received message:     qwered 
Received message:     qwered 
Received message:     qwered 
Received message:     qwered 


Чтение офсетов для конкретной группы консумеров
Для чтения текущих офсетов для группы консумеров можно использовать метод committed(partitions). Этот метод возвращает список офсетов для указанных разделов (partitions).

In [33]:
from confluent_kafka import Consumer, TopicPartition

# Конфигурация консюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my-group11wfewf',
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': False
}

consumer = Consumer(conf)
consumer.subscribe(['my-topic'])

# Получение текущих офсетов для группы консумеров по разделам топика
partitions = [TopicPartition('my-topic', partition=0)]
offsets = consumer.committed(partitions)

for tp in offsets:
    print(f'Partition: {tp.partition}, Offset: {tp.offset}')

consumer.close()


Partition: 0, Offset: 9


### Producer


In [34]:
from confluent_kafka import Producer

# Функция для обработки успешной доставки сообщений
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}')

# Конфигурация продюсера
conf = {
    'bootstrap.servers': 'localhost:9092',  # Адрес Kafka брокера
    'client.id': 'my-producer'                # Идентификатор клиента
}

# Создание экземпляра продюсера
producer = Producer(conf)

# Отправка сообщений
try:
    for i in range(4):
        key = f'key-{i}'                # Ключ сообщения (опционально)
        value = f'Hello Kafka {i}'      # Значение сообщения
        producer.produce('my-topic', key=key, value=value, callback=delivery_report)

        # Обработка асинхронных операций
        producer.poll(0)  # Проверка на наличие ошибок и вызов обратных функций

    # Ожидание отправки всех сообщений
    producer.flush()

except Exception as e:
    print(f'Error while producing: {e}')

finally:
    # Закрытие продюсера
    print('finally')


Message delivered to my-topic [0] at offset 9
Message delivered to my-topic [0] at offset 10
Message delivered to my-topic [0] at offset 11
Message delivered to my-topic [0] at offset 12
finally


### exactly-once semantics, EOS

 idempotent producer и настроек для транзакций
 Конфигурация продюсера:

enable.idempotence: True: Включает идемпотентный режим для продюсера.
transactional.id: Уникальный идентификатор для транзакций. Он должен быть уникальным для каждого продюсера в вашем приложении.

In [36]:
from confluent_kafka import Producer

# Функция для обработки успешной доставки сообщений
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}')

# Конфигурация продюсера
conf = {
    'bootstrap.servers': 'localhost:9092',  # Адрес Kafka брокера
    'client.id': 'my-producer',              # Идентификатор клиента
    'enable.idempotence': True,               # Включение идемпотентного режима
    'acks': 'all',                            # Подтверждение от всех реплик
    'retries': 5,                             # Количество попыток повторной отправки
    'transactional.id': 'my-transactional-id'  # Идентификатор транзакции
}

# Создание экземпляра продюсера
producer = Producer(conf)

# Инициализация транзакции
producer.init_transactions()

try:
    # Начало транзакции
    producer.begin_transaction()

    # Отправка сообщений
    for i in range(4):
        key = f'key-{i}'                # Ключ сообщения (опционально)
        value = f'Hello Kafka {i}'      # Значение сообщения
        producer.produce('my-topic', key=key, value=value, callback=delivery_report)

        # Обработка асинхронных операций
        producer.poll(0)  # Проверка на наличие ошибок и вызов обратных функций

    # Завершение транзакции
    producer.commit_transaction()

except Exception as e:
    print(f'Error while producing: {e}')
    # Отмена транзакции в случае ошибки
    producer.abort_transaction()

finally:
    # Ожидание отправки всех сообщений
    producer.flush()


%4|1729763253.160|GETPID|my-producer#producer-16| [thrd:main]: Failed to acquire transactional PID from broker TxnCoordinator/1: Broker: Not coordinator: retrying


Message delivered to my-topic [0] at offset 13
Message delivered to my-topic [0] at offset 14
Message delivered to my-topic [0] at offset 15
Message delivered to my-topic [0] at offset 16


## Schema registry

In [52]:
import requests
import json

In [55]:

response = requests.get("http://localhost:8081/subjects")
response.json()

[]

In [54]:
### УДАЛЕНИЕ
import requests

# URL для удаления схемы
url = 'http://localhost:8081/subjects/my-avro-topic-value'

# Отправляем DELETE-запрос
response = requests.delete(url)

# Проверяем результат
if response.status_code == 200:
    print(f"Схема для 'my-topic4-value' успешно удалена.")
else:
    print(f"Ошибка при удалении схемы: {response.status_code}\n{response.text}")


Ошибка при удалении схемы: 404
{"error_code":40401,"message":"Subject 'my-avro-topic-value' not found. io.confluent.rest.exceptions.RestNotFoundException: Subject 'my-avro-topic-value' not found.\nio.confluent.rest.exceptions.RestNotFoundException: Subject 'my-avro-topic-value' not found.\n\tat io.confluent.kafka.schemaregistry.rest.exceptions.Errors.subjectNotFoundException(Errors.java:78)\n\tat io.confluent.kafka.schemaregistry.rest.resources.SubjectsResource.deleteSubject(SubjectsResource.java:278)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)\n\tat java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)\n\tat java.base/java.lang.reflect.Method.invoke(Method.java:566)\n\tat org.glassfish.jersey.server.model.internal.ResourceMethodInvocationHandlerFactory.lambda$static$0(ResourceMethodInvocationH

In [23]:
response = requests.get("http://localhost:8081/subjects/my-avro-topic-value/versions/latest")
response.json()

{'error_code': 40401,
 'message': "Subject 'my-avro-topic-value' not found. io.confluent.rest.exceptions.RestNotFoundException: Subject 'my-avro-topic-value' not found.\nio.confluent.rest.exceptions.RestNotFoundException: Subject 'my-avro-topic-value' not found.\n\tat io.confluent.kafka.schemaregistry.rest.exceptions.Errors.subjectNotFoundException(Errors.java:78)\n\tat io.confluent.kafka.schemaregistry.rest.resources.SubjectVersionsResource.getSchemaByVersion(SubjectVersionsResource.java:152)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)\n\tat java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)\n\tat java.base/java.lang.reflect.Method.invoke(Method.java:566)\n\tat org.glassfish.jersey.server.model.internal.ResourceMethodInvocationHandlerFactory.lambda$static$0(ResourceMethodInvocationHandlerFa

Регистрация новой схемы
регистрируем JSON-схему

json_schema = {
    "title": "User",
    "type": "object",
 

In [33]:
import requests
import json

# Конфигурация Schema Registry
schema_registry_url = 'http://localhost:8081'
subject = 'my-json-topic-value'  # Название subject для вашей схемы

# JSON-схема с массивом пользователей
json_schema = {
    "title": "UserList",
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "age": {"type": "integer"},
            "email": {"type": "string"}
        },
        "required": ["name", "age"]
    }
}

# Подготовка данных для регистрации схемы
data = {
    "schema": json.dumps(json_schema),
    "schemaType": "JSON"  # Указываем тип схемы
}

# Запрос на регистрацию схемы
response = requests.post(
    f'{schema_registry_url}/subjects/{subject}/versions',
    headers={'Content-Type': 'application/json'},
    data=json.dumps(data)
)

# Проверка ответа
if response.status_code == 200:
    print(f"Схема успешно зарегистрирована: {response.json()}")
else:
    print(f"Ошибка регистрации схемы: {response.status_code} {response.text}")


Схема успешно зарегистрирована: {'id': 7}


In [34]:
schema_id = 7  # Замените на нужный ID
url = f'http://localhost:8081/schemas/ids/{schema_id}'
response = requests.get(url)

if response.status_code == 200:
    schema = response.json().get('schema')
    print('Полученная схема:', schema)
else:
    print('Ошибка при получении схемы:', response.text)


Полученная схема: {"title":"UserList","type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"age":{"type":"integer"},"email":{"type":"string"}},"required":["name","age"]}}


## пишем JSON используя новую схему

In [25]:
from confluent_kafka import Producer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.json_schema import JSONSerializer
from confluent_kafka.serialization import SerializationContext, MessageField
import json
import uuid

# Конфигурация Schema Registry
schema_registry_conf = {'url': "http://localhost:8081"}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

# Получаем последнюю версию схемы из Schema Registry
subject = 'my-json-topic-value'  # Субъект (subject), под которым схема зарегистрирована
schema_response = schema_registry_client.get_latest_version(subject)
schema_id = schema_response.schema_id
json_schema_str = schema_response.schema.schema_str
print(json_schema_str)

{"title":"UserList","type":"array","items":{"type":"object","properties":{"name":{"type":"string"},"age":{"type":"integer"},"email":{"type":"string"}},"required":["name","age"]}}


In [28]:

# Создаем JSONSerializer с использованием загруженной схемы
json_serializer = JSONSerializer(json_schema_str, schema_registry_client)

# Конфигурация продюсера Kafka
producer_conf = {
    'bootstrap.servers': 'localhost:9092'
}
producer = Producer(producer_conf)

# Топик Kafka
topic = 'my-json-topic'

# Пример JSON-сообщения, которое соответствует схеме
data = [
    {"name": "Alice", "age": 30, "email": "alice@example.com"},
    {"name": "Bob", "age": 25, "email": "bob@example.com"}
]

# Функция для обратного вызова при доставке сообщения
def delivery_report(err, msg):
    if err is not None:
        print(f"Delivery failed for record {msg.key()}: {err}")
    else:
        print(f"Record {msg.key()} successfully produced to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

# Производство сообщений в топик Kafka
try:
    key = str(uuid.uuid4())  # Генерация уникального ключа для каждого сообщения
    serialized_value = json_serializer(data, SerializationContext(topic, MessageField.VALUE))
    
    producer.produce(
        topic=topic,
        key=key,
        value=serialized_value,
        on_delivery=delivery_report
    )
    producer.flush()  # Дождаться отправки всех сообщений
except Exception as e:
    print(f"Error: {e}")

print("Message sent successfully.")


Record b'1bc6fb41-c847-46e1-b214-7228929553d0' successfully produced to my-json-topic [0] at offset 0
Message sent successfully.


In [29]:
## пишем другую схему
# Создаем JSONSerializer с использованием загруженной схемы
json_serializer = JSONSerializer(json_schema_str, schema_registry_client)

# Конфигурация продюсера Kafka
producer_conf = {
    'bootstrap.servers': 'localhost:9092'
}
producer = Producer(producer_conf)

# Топик Kafka
topic = 'my-json-topic'

# Пример JSON-сообщения, которое соответствует схеме
data = [
    {"name1": "Alice", "age": 30, "email": "alice@example.com"},
    {"name2": "Bob", "age": 25, "email": "bob@example.com"}
]

# Функция для обратного вызова при доставке сообщения
def delivery_report(err, msg):
    if err is not None:
        print(f"Delivery failed for record {msg.key()}: {err}")
    else:
        print(f"Record {msg.key()} successfully produced to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

# Производство сообщений в топик Kafka
try:
    key = str(uuid.uuid4())  # Генерация уникального ключа для каждого сообщения
    serialized_value = json_serializer(data, SerializationContext(topic, MessageField.VALUE))
    
    producer.produce(
        topic=topic,
        key=key,
        value=serialized_value,
        on_delivery=delivery_report
    )
    producer.flush()  # Дождаться отправки всех сообщений
except Exception as e:
    print(f"Error: {e}")

print("Message sent successfully.")

Error: 'name' is a required property
Message sent successfully.


AVRO-схема

In [45]:
#пробуем зарегистрировать несколько разновидностей

In [40]:
##
import requests
import json

url = 'http://localhost:8081/subjects/my-topic-value/versions'
schema = {
    "schema": json.dumps({
        "type": "record",
        "name": "User",
        "fields": [
            {"name": "name", "type": "string"},
            {"name": "age", "type": "int"}
            ,
            {"name": "email", "type": "int"}#, "default": ""}
        ]
    })
}
## POST-запрос!!!
response = requests.post(url, json=schema)

if response.status_code == 200:
    schema_id = response.json().get('id')
    print('Схема зарегистрирована с ID:', schema_id)
else:
    print('Ошибка при регистрации схемы:', response.text)

Схема зарегистрирована с ID: 8


In [43]:
response = requests.get("http://localhost:8081/subjects/my-topic-value/versions")
response.json()

[3, 4, 5]

In [46]:
response = requests.get("http://localhost:8081/subjects/my-topic-value/versions/latest")
response.json()

{'subject': 'my-topic-value',
 'version': 5,
 'id': 8,
 'schema': '{"type":"record","name":"User","fields":[{"name":"name","type":"string"},{"name":"age","type":"int"},{"name":"email","type":"int"}]}'}

Для AVRO - Консюмер определит версию схемы по магическим байтикам.

Для JSON надо делать ручками, например - писать версию в само соообщение

AVRO

Avro-файл — это бинарный файл, который используется для хранения данных, сериализованных в формате Avro. Он состоит из двух основных частей:

Заголовок (Header): содержит метаданные и схему данных, которая описывает формат записей в файле.
Тело (Data Blocks): содержит сами данные (записи), закодированные в соответствии с указанной схемой.

In [28]:
#Структура Avro-файла
# 1. Заголовок (Header)
# Каждый Avro-файл начинается с заголовка, который включает в себя:

# Магическое число (magic): первые 4 байта файла — это последовательность байт
#  0x4F 0x62 0x6A 0x01 (или строка "Obj\x01"). Это используется для идентификации файла 
# как Avro-файл.
#Метаданные (Metadata): хранит JSON-объект с информацией о схеме (например, "avro.schema") 
# и другую служебную информацию.
#Синхронизационный маркер (Sync Marker): случайная последовательность из 16 байт, которая используется для синхронизации при чтении файла блоками.

In [24]:
## работа с avro 
import fastavro

# Определение схемы
schema = {
    "type": "record",
    "name": "User",
    "fields": [
        {"name": "name", "type": "string"},
        {"name": "age", "type": "int"}
    ]
}

# Данные, которые будут сериализованы
records = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 30}
]

# Запись данных в Avro-файл
with open('test.avro', 'wb') as out:
    fastavro.writer(out, schema, records)


In [35]:
import fastavro

# Чтение Avro-файла
with open('test.avro', 'rb') as f:
    reader = fastavro.reader(f)
    for record in reader:
        print(record)


{'name': 'Alice', 'age': 25}
{'name': 'Bob', 'age': 30}


In [36]:
from uuid import uuid4
str(uuid4())

'11d10a2c-1188-48bf-9540-0a866c711adb'

#### Пример в kafka_avro_producer.py !
======================================

In [1]:
### Читаем сырые байтики
from confluent_kafka import Consumer

# Конфигурация консьюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my_group11',
    'auto.offset.reset': 'earliest'  # Чтение сообщений с начала, если нет сохранённых оффсетов
}

# Создаем консьюмера
consumer = Consumer(conf)

# Подписка на топик
topic = "my-avro-topic"
consumer.subscribe([topic])

print(f"Listening to topic '{topic}'...")

# Чтение сообщений
try:
    for x in range(10):
        msg = consumer.poll(1.0)  # Ожидание сообщений

        if msg is None:
            continue

        if msg.error():
            print(f"Consumer error: {msg.error()}")
            continue

        # Получение "сырых" данных сообщения
        raw_key = msg.key()  # Байты ключа
        raw_value = msg.value()  # Байты значения

        # Вывод байтов сообщения
        print(f"Raw key (bytes): {raw_key}")
        print(f"Raw value (bytes): {raw_value}")

        # Если хотите вывести их в текстовом виде (для строк):
        print(f"Key (decoded): {raw_key.decode('utf-8') if raw_key else None}")
        print(f"Value (decoded): {raw_value.decode('utf-8') if raw_value else None}")

except KeyboardInterrupt:
    print("Consumer interrupted.")
finally:
    consumer.close()


Listening to topic 'my-avro-topic'...
Raw key (bytes): b'89cd9d5e-6a08-49e5-bafb-162a56f6f95c'
Raw value (bytes): b'\x00\x00\x00\x00\x03\x06qwe\x02\x08red '
Key (decoded): 89cd9d5e-6a08-49e5-bafb-162a56f6f95c
Value (decoded):     qwered 
Raw key (bytes): b'69842775-b927-4ec8-bf3f-8a3a6856de93'
Raw value (bytes): b'\x00\x00\x00\x00\x03\x06qwe\x02\x08red '
Key (decoded): 69842775-b927-4ec8-bf3f-8a3a6856de93
Value (decoded):     qwered 
Raw key (bytes): b'15f44a57-c27f-46fe-8079-0a20f7c11b36'
Raw value (bytes): b'\x00\x00\x00\x00\x03\x06qwe\x02\x08red '
Key (decoded): 15f44a57-c27f-46fe-8079-0a20f7c11b36
Value (decoded):     qwered 


In [79]:
from confluent_kafka import Consumer
from confluent_kafka.avro import AvroConsumer

# Конфигурация консьюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my_group11',
    'auto.offset.reset': 'earliest',  # Начинать чтение с самого начала, если нет смещений
    'schema.registry.url': 'http://localhost:8081',  # Schema Registry URL
}

# Создаем Avro-консьюмера
consumer = AvroConsumer(conf)

# Подписываемся на топик
consumer.subscribe(['my-avro-topic'])

# Чтение сообщений
try:
    for x in range(10):
        msg = consumer.poll(1.0)

        if msg is None:
            continue

        # Выводим полученное сообщение
        print(f"Получено сообщение: {msg.value()}")

except KeyboardInterrupt:
    pass
finally:
    consumer.close()


/tmp/ipykernel_68507/1827773791.py:13: DeprecationWarning: AvroConsumer has been deprecated. Use AvroDeserializer instead.
  consumer = AvroConsumer(conf)


SerializerError: Message deserialization failed for message at my-avro-topic [0] offset 8: message does not start with magic byte

In [80]:
response = requests.get("http://localhost:8081/subjects/my-topic-value/versions/latest")
response.json()

{'error_code': 40401,
 'message': "Subject 'my-topic-value' not found. io.confluent.rest.exceptions.RestNotFoundException: Subject 'my-topic-value' not found.\nio.confluent.rest.exceptions.RestNotFoundException: Subject 'my-topic-value' not found.\n\tat io.confluent.kafka.schemaregistry.rest.exceptions.Errors.subjectNotFoundException(Errors.java:78)\n\tat io.confluent.kafka.schemaregistry.rest.resources.SubjectVersionsResource.getSchemaByVersion(SubjectVersionsResource.java:152)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)\n\tat java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)\n\tat java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)\n\tat java.base/java.lang.reflect.Method.invoke(Method.java:566)\n\tat org.glassfish.jersey.server.model.internal.ResourceMethodInvocationHandlerFactory.lambda$static$0(ResourceMethodInvocationHandlerFactory.java:52)\

### KAFKA CONNECT

In [83]:
import requests
import json

response = requests.get("http://localhost:8083/connectors")
response.json()

['file-source-connector']

In [84]:
response = requests.get("http://localhost:8083/connectors/file-source-connector")
response.json()

{'name': 'file-source-connector',
 'config': {'connector.class': 'FileStreamSource',
  'file': '/tmp/tmp/test-file.txt',
  'tasks.max': '1',
  'poll.interval.ms': '1000',
  'name': 'file-source-connector',
  'topic': 'file-topic'},
 'tasks': [{'connector': 'file-source-connector', 'task': 0}],
 'type': 'source'}

Создаем коннектор для чтения из файла

In [82]:
import requests
import json

url = 'http://localhost:8083/connectors'

headers = {
    'Content-Type': 'application/json'
}

data = {
    "name": "file-source-connector",  # Имя коннектора
    "config": {
        "connector.class": "FileStreamSource",  # Класс коннектора
        "tasks.max": "1",  # Количество задач
        "file": "/tmp/tmp/test-file.txt",  # Путь к файлу, который нужно читать
        "topic": "file-topic",  # Топик, куда отправлять данные
        "poll.interval.ms": "1000"  # Интервал опроса файла (в миллисекундах)
    }
}

response = requests.post(url, headers=headers, data=json.dumps(data))

if response.status_code == 201:
    print('Файловый коннектор успешно создан')
else:
    print('Ошибка при создании файлового коннектора:', response.status_code, response.text)


Файловый коннектор успешно создан


In [85]:
## STATUS
## GET /connectors/{connector_name}/status
response = requests.get("http://localhost:8083/connectors/file-source-connector/status")
response.json()

{'name': 'file-source-connector',
 'connector': {'state': 'RUNNING', 'worker_id': '172.18.0.6:8083'},
 'tasks': [{'id': 0, 'state': 'RUNNING', 'worker_id': '172.18.0.6:8083'}],
 'type': 'source'}

## DEBEZIUM CDC

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import os
import time

pwd = "secret"
uid = "debezium"
server = "localhost"
db = "mydb"
port = "5432"
#
engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:{port}/{db}')

In [55]:
df = pd.read_sql('select * from public.my_table', engine)
df

,id,name
0,1,John Doe
1,2,Alice
2,3,Bob
3,4,John Doe
4,5,Alice
5,6,Bob


In [74]:
# Создайте DataFrame с данными для вставки
data = {'name': ['asd----', 'Alice', 'Bob']}
df = pd.DataFrame(data)

# Вставка данных в my_table
df.to_sql('my_table', engine, if_exists='append', index=False)

3

In [57]:
response = requests.get("http://localhost:8083/")
response.json()

{'version': '2.6.1',
 'commit': '6b2021cd52659cef',
 'kafka_cluster_id': 'MkU3OEVBNTcwNTJENDM2Qk'}

In [69]:
postgres_connector = {
    "name": "my_table-connector",
    "config": {
        "connector.class": "io.debezium.connector.postgresql.PostgresConnector",
        "database.hostname": "postgr",  
        "database.port": "5432",
        "database.user": "debezium",      
        "database.password": "secret", 
        "database.dbname": "mydb",  
        "plugin.name": "pgoutput",
        "database.server.name": "source",
        "key.converter.schemas.enable": "false",
        "value.converter.schemas.enable": "false",
        "transforms": "unwrap",
        "transforms.unwrap.type": "io.debezium.transforms.ExtractNewRecordState",
        "value.converter": "org.apache.kafka.connect.json.JsonConverter",
        "key.converter": "org.apache.kafka.connect.json.JsonConverter",
        "table.include.list": "public.my_table", 
        "slot.name": "dbz_sales_transaction_slot"
    }
}


In [48]:
response = requests.get('http://localhost:8083/connectors/')
response.json()

['file-source-connector']

%3|1729853781.469|FAIL|rdkafka#producer-6| [thrd:185.12.94.232:9092/1]: 185.12.94.232:9092/1: Connect to ipv4#185.12.94.232:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1729853782.470|FAIL|rdkafka#producer-6| [thrd:185.12.94.232:9092/1]: 185.12.94.232:9092/1: Connect to ipv4#185.12.94.232:9092 failed: Connection refused (after 0ms in state CONNECT, 1 identical error(s) suppressed)
%3|1729853783.470|FAIL|rdkafka#producer-6| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1729853786.471|FAIL|rdkafka#producer-6| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv6#[::1]:9092 failed: Connection refused (after 0ms in state CONNECT)
%3|1729853787.471|FAIL|rdkafka#producer-6| [thrd:localhost:9092/bootstrap]: localhost:9092/bootstrap: Connect to ipv4#127.0.0.1:9092 failed: Connection refused (after 0ms in state CONNECT)
%6|1729853792.488|FAIL|rdkafka#pro

In [47]:
url = 'http://localhost:8083/connectors/'


# Отправляем данные как JSON
response = requests.post(url, json = postgres_connector)

# Проверяем статус код ответа
if response.status_code == 200:
    print("Данные успешно отправлены.")
    print("Ответ сервера:", response.json())  # Если сервер возвращает JSON-ответ
else:
    print("Ошибка при отправке данных:", response.status_code, response.text)

NameError: name 'postgres_connector' is not defined

In [72]:
response = requests.get('http://localhost:8083/connectors/my_table-connector')
response.json()

{'name': 'my_table-connector',
 'config': {'connector.class': 'io.debezium.connector.postgresql.PostgresConnector',
  'database.user': 'debezium',
  'database.dbname': 'mydb',
  'slot.name': 'dbz_sales_transaction_slot',
  'transforms': 'unwrap',
  'database.server.name': 'source',
  'database.port': '5432',
  'plugin.name': 'pgoutput',
  'key.converter.schemas.enable': 'false',
  'database.hostname': 'postgr',
  'database.password': 'secret',
  'value.converter.schemas.enable': 'false',
  'name': 'my_table-connector',
  'transforms.unwrap.type': 'io.debezium.transforms.ExtractNewRecordState',
  'value.converter': 'org.apache.kafka.connect.json.JsonConverter',
  'table.include.list': 'public.my_table',
  'key.converter': 'org.apache.kafka.connect.json.JsonConverter'},
 'tasks': [{'connector': 'my_table-connector', 'task': 0}],
 'type': 'source'}

In [73]:
response = requests.get('http://localhost:8083/connectors/my_table-connector/status')
response.json()

{'name': 'my_table-connector',
 'connector': {'state': 'RUNNING', 'worker_id': '172.18.0.5:8083'},
 'tasks': [{'id': 0, 'state': 'RUNNING', 'worker_id': '172.18.0.5:8083'}],
 'type': 'source'}

In [2]:
!pip show confluent-kafka

Name: confluent-kafka
Version: 2.6.0
Summary: Confluent's Python client for Apache Kafka
Home-page: https://github.com/confluentinc/confluent-kafka-python
Author: Confluent Inc
Author-email: support@confluent.io
License: 
Location: /root/myenv/lib/python3.12/site-packages
Requires: 
Required-by: 


In [64]:
from confluent_kafka.admin import AdminClient   

# Определите адреса брокеров Kafka
bootstrap_servers = ['localhost:9092']

# Настройка Kafka
config = {
    'bootstrap.servers': 'localhost:9092'  # Адрес вашего Kafka-брокера
}
# Создайте экземпляр KafkaAdminClient
admin_client = AdminClient(config)

# Запрос метаданных для получения списка топиков
metadata = admin_client.list_topics(timeout=10)

# Получение списка топиков
topics = metadata.topics

# Вывод списка топиков
for topic in topics:
    print(topic)

connect_status
connect_offsets
connect_configs
_schemas
source.public.my_table
__consumer_offsets


In [ ]:
from kafka import KafkaConsumer
bootstrap_servers = ['localhost:29092']
consumer = KafkaConsumer( bootstrap_servers=bootstrap_servers)
consumer.topics()

In [ ]:
import json
topicName = 'source.public.my_table'
# Initialize consumer variable
consumer = KafkaConsumer (topicName , auto_offset_reset='earliest', 
                          bootstrap_servers = bootstrap_servers, group_id='sales-transactions')

# Read and print message from consumer
for msg in consumer:
    print(json.loads(msg.value))

In [77]:
# Удаление
#response = requests.delete('http://localhost:8083/connectors/my_table-connector')
#response.json()